### Imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import timm
from model import LatenViTSmall
import os
from torch.utils.data import DataLoader, Dataset, ConcatDataset
from PIL import Image
from collections import OrderedDict

c:\Users\Haseeb\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Configuration

In [2]:

class Config:
    MODEL_NAME   = 'vit_small_patch16_224.augreg_in21k'
    NUM_CLASSES  = 7      
    NREPEAT      = 3
    START_BLOCK  = 4
    END_BLOCK    = 9
    NUM_LAYERS   = 11
    
    BATCH_SIZE   = 64
    NUM_EPOCHS   = 5
    LEARNING_RATE= 1e-4
    DEVICE       = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    DATA_ROOT    = "../../../pacs_data/pacs_data"
    DOMAINS      = ["art_painting", "cartoon", "photo", "sketch"]
    
    TRANSFORM = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5,), std=(0.5,))
])


### PACS Dataset Class

In [3]:
class PACSDataset(Dataset):
    def __init__(self, root_dir, domain, transform=None):
        self.root_dir    = os.path.join(root_dir, domain)
        self.transform   = transform
        self.classes     = sorted(os.listdir(self.root_dir))
        self.class_to_idx= {cls_name: i for i, cls_name in enumerate(self.classes)}
        self.images      = []
        self.labels      = []
        
        for cls_name in self.classes:
            cls_dir = os.path.join(self.root_dir, cls_name)
            for img_name in os.listdir(cls_dir):
                self.images.append(os.path.join(cls_dir, img_name))
                self.labels.append(self.class_to_idx[cls_name])
                
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path = self.images[idx]
        image    = Image.open(img_path).convert('RGB')
        label    = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        return image, label


### Model Setup

In [4]:
def setup_model():
    base_model = timm.create_model(Config.MODEL_NAME, pretrained=True)
    model = LatenViTSmall(
        model     = base_model,
        nrepeat   = Config.NREPEAT,
        start     = Config.START_BLOCK,
        end       = Config.END_BLOCK,
        num_layers= Config.NUM_LAYERS
    )
    return model.to(Config.DEVICE)

def setup_baseline_model():
    base_model = timm.create_model(Config.MODEL_NAME, pretrained=True)
    return base_model.to(Config.DEVICE)


### Training Function

In [5]:
def train_epoch(model, dataloader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    correct      = 0
    total        = 0
    
    for images, labels in dataloader:
        images, labels = images.to(Config.DEVICE), labels.to(Config.DEVICE)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss    = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted  = outputs.max(1)
        total        += labels.size(0)
        correct      += predicted.eq(labels).sum().item()
        
    epoch_loss = running_loss / len(dataloader)
    epoch_acc  = 100.0 * correct / total
    return epoch_loss, epoch_acc


### Training Function

In [6]:
@torch.no_grad()
def evaluate(model, dataloader):
    model.eval()
    correct = 0
    total   = 0
    
    for images, labels in dataloader:
        images, labels = images.to(Config.DEVICE), labels.to(Config.DEVICE)
        outputs        = model(images)
        
        _, predicted = outputs.max(1)
        total       += labels.size(0)
        correct     += predicted.eq(labels).sum().item()
        
    return 100.0 * correct / total


### Baseline -CotFormer

In [7]:

train_loaders = []
test_loaders  = []
for domain in Config.DOMAINS:
    ds_train = PACSDataset(Config.DATA_ROOT, domain, Config.TRANSFORM)
    ds_test  = PACSDataset(Config.DATA_ROOT, domain, Config.TRANSFORM)
    train_loaders.append(DataLoader(ds_train, batch_size=Config.BATCH_SIZE, shuffle=True))
    test_loaders .append(DataLoader(ds_test,  batch_size=Config.BATCH_SIZE, shuffle=False))


full_train_ds = ConcatDataset([dl.dataset for dl in train_loaders])
full_test_ds  = ConcatDataset([dl.dataset  for dl in test_loaders ])
full_train_loader = DataLoader(full_train_ds, batch_size=Config.BATCH_SIZE, shuffle=True)
full_test_loader  = DataLoader(full_test_ds,  batch_size=Config.BATCH_SIZE, shuffle=False)

model     = setup_model()
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=Config.LEARNING_RATE)

for epoch in range(1, Config.NUM_EPOCHS + 1):
    loss, acc = train_epoch(model, full_train_loader, criterion, optimizer)
    print(f"[Epoch {epoch}/{Config.NUM_EPOCHS}] Train Loss: {loss:.4f}, Train Acc: {acc:.2f}%")

test_acc = evaluate(model, full_test_loader)
print(f"Baseline (all domains) Test Accuracy: {test_acc:.2f}%")

print("\n[Per-Domain Evaluation]")
domain_accuracies = OrderedDict()
for domain, loader in zip(Config.DOMAINS, test_loaders):
    acc = evaluate(model, loader)
    domain_accuracies[domain] = acc
    print(f"  {domain:>12}: {acc:.2f}%")

[Epoch 1/5] Train Loss: 1.7424, Train Acc: 37.70%
[Epoch 2/5] Train Loss: 0.7537, Train Acc: 71.26%
[Epoch 3/5] Train Loss: 0.4456, Train Acc: 83.45%
[Epoch 4/5] Train Loss: 0.2622, Train Acc: 90.59%
[Epoch 5/5] Train Loss: 0.1460, Train Acc: 94.99%
Baseline (all domains) Test Accuracy: 96.82%

[Per-Domain Evaluation]
  art_painting: 97.66%
       cartoon: 98.93%
         photo: 98.74%
        sketch: 94.30%


In [8]:

transform = Config.TRANSFORM  

lodo_results = OrderedDict()

for test_domain in Config.DOMAINS:
    print(f"\n=== LODO: Held-Out Domain = {test_domain} ===")
    train_loaders = []
    for d in Config.DOMAINS:
        if d == test_domain:
            continue
        ds_train = PACSDataset(Config.DATA_ROOT, d, transform)
        train_loaders.append(DataLoader(ds_train, batch_size=Config.BATCH_SIZE, shuffle=True))

    train_ds = ConcatDataset([dl.dataset for dl in train_loaders])
    train_loader = DataLoader(train_ds, batch_size=Config.BATCH_SIZE, shuffle=True)

    ds_test    = PACSDataset(Config.DATA_ROOT, test_domain, transform)
    test_loader = DataLoader(ds_test, batch_size=Config.BATCH_SIZE, shuffle=False)

    model     = setup_model()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=Config.LEARNING_RATE)

    for epoch in range(1, Config.NUM_EPOCHS + 1):
        loss, acc = train_epoch(model, train_loader, criterion, optimizer)
        print(f"[Epoch {epoch}/{Config.NUM_EPOCHS}] Train Loss: {loss:.4f}, Train Acc: {acc:.2f}%")

    test_acc = evaluate(model, test_loader)
    lodo_results[test_domain] = test_acc
    print(f"--> Test Accuracy on {test_domain}: {test_acc:.2f}%")

print("\n=== LODO Summary ===")
for domain, acc in lodo_results.items():
    print(f"{domain:>14}: {acc:.2f}%")



=== LODO: Held-Out Domain = art_painting ===
[Epoch 1/5] Train Loss: 1.7974, Train Acc: 39.66%
[Epoch 2/5] Train Loss: 0.7279, Train Acc: 71.48%
[Epoch 3/5] Train Loss: 0.4400, Train Acc: 83.38%
[Epoch 4/5] Train Loss: 0.2528, Train Acc: 90.43%
[Epoch 5/5] Train Loss: 0.1392, Train Acc: 94.79%
--> Test Accuracy on art_painting: 48.93%

=== LODO: Held-Out Domain = cartoon ===
[Epoch 1/5] Train Loss: 2.0391, Train Acc: 28.68%
[Epoch 2/5] Train Loss: 0.9724, Train Acc: 62.43%
[Epoch 3/5] Train Loss: 0.5553, Train Acc: 78.88%
[Epoch 4/5] Train Loss: 0.3401, Train Acc: 87.73%
[Epoch 5/5] Train Loss: 0.1453, Train Acc: 94.73%
--> Test Accuracy on cartoon: 49.15%

=== LODO: Held-Out Domain = photo ===
[Epoch 1/5] Train Loss: 1.7979, Train Acc: 36.55%
[Epoch 2/5] Train Loss: 0.8164, Train Acc: 68.98%
[Epoch 3/5] Train Loss: 0.4929, Train Acc: 81.91%
[Epoch 4/5] Train Loss: 0.3162, Train Acc: 89.24%
[Epoch 5/5] Train Loss: 0.1256, Train Acc: 95.45%
--> Test Accuracy on photo: 66.35%

=== LODO:

### Baseline Vanilla

In [7]:

train_loaders = []
test_loaders  = []
for domain in Config.DOMAINS:
    ds_train = PACSDataset(Config.DATA_ROOT, domain, Config.TRANSFORM)
    ds_test  = PACSDataset(Config.DATA_ROOT, domain, Config.TRANSFORM)
    train_loaders.append(DataLoader(ds_train, batch_size=Config.BATCH_SIZE, shuffle=True))
    test_loaders .append(DataLoader(ds_test,  batch_size=Config.BATCH_SIZE, shuffle=False))


full_train_ds = ConcatDataset([dl.dataset for dl in train_loaders])
full_test_ds  = ConcatDataset([dl.dataset  for dl in test_loaders ])
full_train_loader = DataLoader(full_train_ds, batch_size=Config.BATCH_SIZE, shuffle=True)
full_test_loader  = DataLoader(full_test_ds,  batch_size=Config.BATCH_SIZE, shuffle=False)

model     = setup_baseline_model()
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=Config.LEARNING_RATE)

for epoch in range(1, Config.NUM_EPOCHS + 1):
    loss, acc = train_epoch(model, full_train_loader, criterion, optimizer)
    print(f"[Epoch {epoch}/{Config.NUM_EPOCHS}] Train Loss: {loss:.4f}, Train Acc: {acc:.2f}%")

test_acc = evaluate(model, full_test_loader)
print(f"Baseline (all domains) Test Accuracy: {test_acc:.2f}%")

print("\n[Per-Domain Evaluation]")
domain_accuracies = OrderedDict()
for domain, loader in zip(Config.DOMAINS, test_loaders):
    acc = evaluate(model, loader)
    domain_accuracies[domain] = acc
    print(f"  {domain:>12}: {acc:.2f}%")

[Epoch 1/5] Train Loss: 0.7017, Train Acc: 83.62%
[Epoch 2/5] Train Loss: 0.0909, Train Acc: 96.99%
[Epoch 3/5] Train Loss: 0.0221, Train Acc: 99.33%
[Epoch 4/5] Train Loss: 0.0264, Train Acc: 99.25%
[Epoch 5/5] Train Loss: 0.0359, Train Acc: 98.73%
Baseline (all domains) Test Accuracy: 98.46%

[Per-Domain Evaluation]
  art_painting: 98.54%
       cartoon: 99.10%
         photo: 99.88%
        sketch: 97.43%


In [8]:

transform = Config.TRANSFORM  

lodo_results = OrderedDict()

for test_domain in Config.DOMAINS:
    print(f"\n=== LODO: Held-Out Domain = {test_domain} ===")
    train_loaders = []
    for d in Config.DOMAINS:
        if d == test_domain:
            continue
        ds_train = PACSDataset(Config.DATA_ROOT, d, transform)
        train_loaders.append(DataLoader(ds_train, batch_size=Config.BATCH_SIZE, shuffle=True))

    train_ds = ConcatDataset([dl.dataset for dl in train_loaders])
    train_loader = DataLoader(train_ds, batch_size=Config.BATCH_SIZE, shuffle=True)

    ds_test    = PACSDataset(Config.DATA_ROOT, test_domain, transform)
    test_loader = DataLoader(ds_test, batch_size=Config.BATCH_SIZE, shuffle=False)

    model     = setup_baseline_model()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=Config.LEARNING_RATE)

    for epoch in range(1, Config.NUM_EPOCHS + 1):
        loss, acc = train_epoch(model, train_loader, criterion, optimizer)
        print(f"[Epoch {epoch}/{Config.NUM_EPOCHS}] Train Loss: {loss:.4f}, Train Acc: {acc:.2f}%")

    test_acc = evaluate(model, test_loader)
    lodo_results[test_domain] = test_acc
    print(f"--> Test Accuracy on {test_domain}: {test_acc:.2f}%")

print("\n=== LODO Summary ===")
for domain, acc in lodo_results.items():
    print(f"{domain:>14}: {acc:.2f}%")



=== LODO: Held-Out Domain = art_painting ===
[Epoch 1/5] Train Loss: 0.3349, Train Acc: 88.42%
[Epoch 2/5] Train Loss: 0.0540, Train Acc: 98.15%
[Epoch 3/5] Train Loss: 0.0390, Train Acc: 98.55%
[Epoch 4/5] Train Loss: 0.0187, Train Acc: 99.38%
[Epoch 5/5] Train Loss: 0.0270, Train Acc: 99.04%
--> Test Accuracy on art_painting: 87.70%

=== LODO: Held-Out Domain = cartoon ===
[Epoch 1/5] Train Loss: 0.3303, Train Acc: 88.20%
[Epoch 2/5] Train Loss: 0.0664, Train Acc: 97.80%
[Epoch 3/5] Train Loss: 0.0289, Train Acc: 99.02%
[Epoch 4/5] Train Loss: 0.0302, Train Acc: 98.95%
[Epoch 5/5] Train Loss: 0.0288, Train Acc: 99.01%
--> Test Accuracy on cartoon: 79.65%

=== LODO: Held-Out Domain = photo ===
[Epoch 1/5] Train Loss: 0.3755, Train Acc: 86.64%
[Epoch 2/5] Train Loss: 0.0635, Train Acc: 97.98%
[Epoch 3/5] Train Loss: 0.0301, Train Acc: 98.97%
[Epoch 4/5] Train Loss: 0.0244, Train Acc: 99.24%
[Epoch 5/5] Train Loss: 0.0319, Train Acc: 98.93%
--> Test Accuracy on photo: 96.83%

=== LODO: